In [1]:
import pandas as pd
import numpy as np

tickers = ['AAPL', 'AMZN', 'GOOGL', 'MSFT', 'TSLA']
dfs = []

## Dataset Access
After downloading the dataset, we put the data files in the same folder as the .ipynb file. That is, in the project we have,
* notebook1.ipynb
* aapl.us.txt
* amzn.us.txt
* googl.us.txt
* msft.us.txt
* tsla.us.txt

In [2]:
for ticker in tickers:
    df = pd.read_csv(f'{ticker.lower()}.us.txt')
    df['Ticker'] = ticker
    df['Date'] = pd.to_datetime(df['Date'])
    dfs.append(df)

* Here, we create a list of dataframes so that they can all be merged into a single multi-indexed dataframe
* The `Date` column is converted to `datetime` format

In [3]:
df = pd.concat(dfs)
df.set_index(['Ticker', 'Date'], inplace = True)
df.head()

Open     High      Low    Close    Volume  OpenInt
Ticker Date                                                             
AAPL   1984-09-07  0.42388  0.42902  0.41874  0.42388  23220030        0
       1984-09-10  0.42388  0.42516  0.41366  0.42134  18022532        0
       1984-09-11  0.42516  0.43668  0.42516  0.42902  42498199        0
       1984-09-12  0.42902  0.43157  0.41618  0.41618  37125801        0
       1984-09-13  0.43927  0.44052  0.43927  0.43927  57822062        0

# Data Cleaning

## 1. Handling Missing Values
We use `df.isnull().sum()` to find the number of null values in each column. It turns out to be zero, so we do not have to remove any rows or fill them by interpolation.

## 2. Converting Date to datetime format (already done)

## 3. Filter the data to include only the last 10 years
The final date for which data exists is 2017-11-10 (yyyy-mm-dd format), so by using a cutoff date 10 years before that, we only take in the values that are present in that 10 year period and none before it.

In [4]:
df.isnull().sum()

Open       0
High       0
Low        0
Close      0
Volume     0
OpenInt    0
dtype: int64

In [5]:
df.sort_index(inplace = True)
cutoff_date = pd.to_datetime('2007-11-10')
df = df[df.index.get_level_values('Date') >= cutoff_date]
df.head()

Open    High     Low   Close     Volume  OpenInt
Ticker Date                                                          
AAPL   2007-11-12  21.130  21.479  19.291  19.691  492362604        0
       2007-11-13  20.615  21.897  19.691  21.765  484373501        0
       2007-11-14  22.733  22.739  20.970  21.274  403585172        0
       2007-11-15  21.280  21.717  20.528  21.040  414487458        0
       2007-11-16  21.193  21.388  20.405  21.309  385660112        0

## Data Transformation

In [6]:
df['Daily Return'] = np.nan
df['7D MA'] = np.nan
df['30D MA'] = np.nan
df['30D Rol. Vol.'] = np.nan

In [7]:
for ticker in tickers:
    df_ticker = df.loc[ticker].copy()

    df_ticker['Daily Return'] = df_ticker['Close'].pct_change() * 100
    df_ticker['7D MA'] = df_ticker['Close'].rolling(window = 7).mean()
    df_ticker['30D MA'] = df_ticker['Close'].rolling(window = 30).mean()
    df_ticker['30D Rol. Vol.'] = df_ticker['Daily Return'].rolling(window = 30).std()

    df_ticker['Ticker'] = ticker
    df_ticker.set_index('Ticker', append = True, inplace = True)
    df_ticker = df_ticker.reorder_levels(['Ticker', 'Date'])

    df.update(df_ticker[['Daily Return', '7D MA', '30D MA', '30D Rol. Vol.']])

df.head(31)

Open    High     Low   Close     Volume  OpenInt  \
Ticker Date                                                             
AAPL   2007-11-12  21.130  21.479  19.291  19.691  492362604        0   
       2007-11-13  20.615  21.897  19.691  21.765  484373501        0   
       2007-11-14  22.733  22.739  20.970  21.274  403585172        0   
       2007-11-15  21.280  21.717  20.528  21.040  414487458        0   
       2007-11-16  21.193  21.388  20.405  21.309  385660112        0   
       2007-11-19  21.262  21.540  20.758  20.996  321694225        0   
       2007-11-20  21.247  22.000  20.942  21.623  429930531        0   
       2007-11-21  21.195  22.072  21.086  21.572  339509885        0   
       2007-11-23  22.027  22.033  21.739  21.967  129891462        0   
       2007-11-26  22.268  22.700  22.072  22.097  363869639        0   
       2007-11-27  22.439  22.511  21.772  22.387  366923874        0   
       2007-11-28  22.669  23.128  22.456  23.080  320237489        0   
       2007-11-29  22.959  23.714  22.943  23.600  291924936        0   
       2007-11-30  24.016  24.039  23.012  23.335  331057965        0   
       2007-12-03  23.282  23.584  22.755  22.908  267838964        0   
       2007-12-04  22.679  23.167  22.667  23.027  215691856        0   
       2007-12-05  23.435  23.819  23.361  23.755  248378681        0   
       2007-12-06  23.847  24.346  23.834  24.327  250866121        0   
       2007-12-07  24.401  24.972  24.081  24.883  297123533        0   
       2007-12-10  24.790  25.057  24.675  24.873  201210463        0   
       2007-12-11  24.947  25.207  23.996  24.145  308945448        0   
       2007-12-12  24.779  24.905  23.787  24.441  341155751        0   
       2007-12-13  24.346  24.604  24.053  24.565  241126701        0   
       2007-12-14  24.369  25.612  24.273  24.381  188026204        0   
       2007-12-17  24.423  24.672  23.433  23.615  285267629        0   
       2007-12-18  23.916  23.989  22.872  23.433  340716873        0   
       2007-12-19  23.447  23.644  23.167  23.451  230164023        0   
       2007-12-20  23.748  24.054  23.477  23.974  215053709        0   
       2007-12-21  24.336  24.833  24.319  24.833  277136695        0   
       2007-12-24  24.973  25.527  24.944  25.459  133938457        0   
       2007-12-26  25.486  25.735  25.205  25.478  196023777        0   

                   Daily Return      7D MA   30D MA  30D Rol. Vol.  
Ticker Date                                                         
AAPL   2007-11-12           NaN        NaN      NaN            NaN  
       2007-11-13     10.532731        NaN      NaN            NaN  
       2007-11-14     -2.255915        NaN      NaN            NaN  
       2007-11-15     -1.099934        NaN      NaN            NaN  
       2007-11-16      1.278517        NaN      NaN            NaN  
       2007-11-19     -1.468863        NaN      NaN            NaN  
       2007-11-20      2.986283  21.099714      NaN            NaN  
       2007-11-21     -0.235860  21.368429      NaN            NaN  
       2007-11-23      1.831077  21.397286      NaN            NaN  
       2007-11-26      0.591797  21.514857      NaN            NaN  
       2007-11-27      1.312395  21.707286      NaN            NaN  
       2007-11-28      3.095547  21.960286      NaN            NaN  
       2007-11-29      2.253033  22.332286      NaN            NaN  
       2007-11-30     -1.122881  22.576857      NaN            NaN  
       2007-12-03     -1.829869  22.767714      NaN            NaN  
       2007-12-04      0.519469  22.919143      NaN            NaN  
       2007-12-05      3.161506  23.156000      NaN            NaN  
       2007-12-06      2.407914  23.433143      NaN            NaN  
       2007-12-07      2.285526  23.690714      NaN            NaN  
       2007-12-10     -0.040188  23.872571      NaN            NaN  
       2007-12-11     -2.926868  23.988286      NaN            NaN  
       2007-12-12      1

In [19]:
print('Ticker', 'Avg Daily Return', 'Max Volatility', 'Volatility Date')

for ticker in tickers:
    df_ticker = df.loc[ticker].copy()
    avg_daily_return = df_ticker['Daily Return'].mean()
    max_vol = df_ticker['30D Rol. Vol.'].max()
    max_vol_date = df_ticker['30D Rol. Vol.'].idxmax()
    
    print(ticker, avg_daily_return, max_vol, max_vol_date)

Ticker Avg Daily Return Max Volatility Volatility Date
AAPL 0.10637467108104491 6.69549168416663 2008-10-28 00:00:00
AMZN 0.1359324973834691 6.942334931099093 2008-10-28 00:00:00
GOOGL 0.06445391675026987 5.552183775675065 2008-10-28 00:00:00
MSFT 0.05893771093946893 6.081447547876392 2008-11-21 00:00:00
TSLA 0.21008256936590616 9.445034250061946 2010-08-10 00:00:00


## Exploratory Data Analysis
We can see that the highest average daily returns over the 10 year period are for Tesla at about 0.21%.
The most volatile 30 day period occurs for Tesla ending on 2010-08-10, where the 30 day rolling volatility is 9.44